In [1]:
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

dill 0.3.8 is installed but dill<0.3.2,>=0.3.1.1 is required
Redoing requirement with just package name...
cloudpickle 3.0.0 is installed but cloudpickle~=2.2.1 is required
Redoing requirement with just package name...
numpy 1.26.4 is installed but numpy<1.25.0,>=1.14.3 is required
Redoing requirement with just package name...
pyarrow 16.1.0 is installed but pyarrow<10.0.0,>=3.0.0 is required
Redoing requirement with just package name...
jupyterlab 4.2.5 is installed but jupyterlab~=3.6.0 is required
Redoing requirement with just package name...
google-cloud-bigquery 2.34.4 is installed but google-cloud-bigquery[bqstorage,pandas]>=3.10.0 is required
Redoing requirement with just package name...
google-cloud-storage 1.44.0 is installed but google-cloud-storage>=2.0.0 is required
Redoing requirement with just package name...
pandas 2.2.2 is installed but pandas<2.1.4,>=1.5.0 is required
Redoing requirement with just package name...
botocore 1.35.23 is installed but botocore<1.30.0,>=1.29

In [2]:
import wandb
from huggingface_hub import login

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("token_hf")
wanb_token = user_secrets.get_secret("token_wanb")

# This will prompt you to enter your Hugging Face token in a notebook-friendly way
login(token = hf_token)
wandb.login(key = wanb_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch
from unsloth import FastLanguageModel

max_seq_length = 5096 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.7 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [5]:
from datasets import load_dataset
dataset = load_dataset("vijay-ravichander/cnn_databricks_summarization", split="train")

README.md:   0%|          | 0.00/501 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6559 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1640 [00:00<?, ? examples/s]

In [6]:
def formatting_prompts_func(example):
    messages = [
    {"role": "user", "content": example['instruction'] + "\n" + example["context"] + "\n" + "Given the instruction and context. Generate a summary"},
    {"role": "assistant", "content": example['response']},
    ]
    texts = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
    return { "task" : texts}

# Applying Chat Template to Dataset and Not Tokenizing
dataset = dataset.map(formatting_prompts_func, batched= False)

Map:   0%|          | 0/6559 [00:00<?, ? examples/s]

In [7]:
def tokenize_function(example):
    outputs = tokenizer(example["task"], truncation=True, padding="max_length", max_length=5096)
    return outputs
    
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns = ["instruction", "context", "response" ], load_from_cache_file=False)
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/6559 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets

Dataset({
    features: ['task', 'input_ids', 'attention_mask'],
    num_rows: 6559
})

In [9]:
trained_model_name = "Llama-1B-Summarization-LoRA-All-r128"
huggingface_username = "vijay-ravichander"

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_datasets,
    dataset_text_field = "task",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = True,
        bf16 = False,
        logging_steps = 20,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        save_strategy="steps",  # Choose "steps" or "epoch"
        save_steps=200,
        output_dir = "outputs",
        report_to = "wandb",  # Reporting to Weights & Biases
        push_to_hub=True,  # Push the model to Hugging Face Hub
        hub_model_id= f"{huggingface_username}/{trained_model_name}",  # Set this to your Hugging Face model name
        hub_strategy="all_checkpoints"
    ),
)

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [11]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/6559 [00:00<?, ? examples/s]

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,559 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 820
 "-____-"     Number of trainable parameters = 90,177,536
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: vijayravichander. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241118_170847-7w7hgeeo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run outputs
wandb: ⭐️ View project at https://wandb.ai/vijayravichander/huggingface
wandb: 🚀 View run at https://wandb.ai/vijayravichander/huggingface/runs/7w7hgeeo


Step,Training Loss
20,2.606200
40,2.325800
60,2.327000
80,2.270700
100,2.244100
120,2.197100
140,2.296800
160,2.188000
180,2.190300
200,2.246400


In [13]:
model.save_pretrained(f"/kaggle/working/{trained_model_name}") 
tokenizer.save_pretrained(f"/kaggle/working/{trained_model_name}")

('/kaggle/working/Llama-1B-Summarization-LoRA-All-r128/tokenizer_config.json',
 '/kaggle/working/Llama-1B-Summarization-LoRA-All-r128/special_tokens_map.json',
 '/kaggle/working/Llama-1B-Summarization-LoRA-All-r128/tokenizer.json')

In [14]:
model.push_to_hub(trained_model_name) # Online saving
tokenizer.push_to_hub(trained_model_name) # Online saving

Saved model to https://huggingface.co/Llama-1B-Summarization-LoRA-All-r128


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [15]:
merged_model_name = f"{huggingface_username}/{trained_model_name}-merged"
model.push_to_hub_merged(merged_model_name, tokenizer)

Unsloth: You are pushing to hub in Kaggle environment.
To save memory, we shall move vijay-ravichander/Llama-1B-Summarization-LoRA-All-r128-merged to /tmp/Llama-1B-Summarization-LoRA-All-r128-merged
Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.5G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 18.93 out of 31.35 RAM for saving.


100%|██████████| 16/16 [00:00<00:00, 45.77it/s]


Unsloth: Saving tokenizer...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

 Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving /tmp/Llama-1B-Summarization-LoRA-All-r128-merged/pytorch_model.bin...


pytorch_model.bin:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/vijay-ravichander/Llama-1B-Summarization-LoRA-All-r128-merged


In [16]:
#Inference
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

new_model_name = merged_model_name
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = new_model_name, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", 
     "content": '''
     Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a 
     midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a 
     regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the 
     J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the 
     summer. In 2007, he moved to the Japan Football League club Rosso Kumamoto (later Roasso Kumamoto) based in his local region. He played as a regular player and the club was 
     promoted to J2 in 2008. Although he did not play as much, he still played in many matches. In 2010, he moved to Indonesia and joined Persela Lamongan. In July 2010, he returned to 
     Japan and joined the J2 club Giravanz Kitakyushu. He played often as a defensive midfielder and center back until 2012 when he retired.'''},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


pytorch_model.bin:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 18 Nov 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nKomorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a \n     midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a \n     regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the \n     J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the \n     summer. In 2007, he moved to the Japan Football League club Rosso Kumamoto (later Ro